In [1]:

# importing old libraries (also used in the two first parts)
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs

In [2]:
# importing new libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Load the pandas dataframe (created in part 2 of the assignment)

In [3]:
df_toronto = pd.read_csv('toronto_base.csv')
df_toronto.head(7)

,Unnamed: 0,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,0,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
1,1,M4H,East York,Thorncliffe Park,43.705369,-79.349372
2,2,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
3,3,M9W,Etobicoke,Northwest,43.706748,-79.594054
4,4,M9L,North York,Humber Summit,43.756303,-79.565963
5,5,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
6,6,M9N,York,Weston,43.706876,-79.518188


### Create a map of Toronto

In [4]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

### Create a new data frame with neighborhoods in Scarborough

In [5]:
# @hiddel_cell
CLIENT_ID = 'N2EQLVXLCWFDXBTEA42MHUVILYIEAZR1ZGRS3RCWJPZ3CI5D' # your Foursquare ID
CLIENT_SECRET = 'FJRT1IXT0USP20LMD0YWDORQF0RY3IJK2GHMVIBTBWHCME4X' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [6]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(7)

,Unnamed: 0,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,2,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,11,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302
2,13,M1W,Scarborough,Steeles West,43.799525,-79.318389
3,32,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
4,33,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476
5,39,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
6,42,M1R,Scarborough,"Maryvale, Wexford",43.750071,-79.295849


### Create a map of Scarborough and its neighbourhoods

In [7]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [8]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

### Get the top 100 venues in the neighborhood 'Steeles West', from Scarborough

In [9]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Rouge, Malvern" are 43.8066863, -79.19435340000003.


In [10]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c7cd417f594df76bb3589a2'},
 'response': {'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 45,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775537,
          'lng': -79.256833}],
        'distance

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
2,DAVIDsTEA,Tea Room,43.776613,-79.258516
3,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
4,SEPHORA,Cosmetics Shop,43.775592,-79.258242
5,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
6,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
7,Scarborough Town Centre,Shopping Mall,43.775545,-79.257545
8,Moxie's Classic Grill,American Restaurant,43.776219,-79.256820
9,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

45 venues were returned by Foursquare.


In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get venues for each neighborhood in Scarborough

In [16]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge, Malvern
Tam O'Shanter
Steeles West
Upper Rouge
Cliffcrest, Cliffside
Highland Creek, Rouge Hill, Port Union
Maryvale, Wexford
Scarborough Village
Agincourt North, Milliken
Clairlea, Golden Mile, Oakridge
Woburn
Ionview, Kennedy Park
Birch Cliff
Morningside, West Hill
Cedarbrae
Agincourt
Dorset Park, Scarborough Town Centre, Wexford Heights


In [17]:
scarborough_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Tam O'Shanter,43.781638,-79.304302,Remezzo Italian Bistro,43.778649,-79.308264,Italian Restaurant
2,Tam O'Shanter,43.781638,-79.304302,The Royal Chinese Restaurant 避風塘小炒,43.780505,-79.298844,Chinese Restaurant
3,Tam O'Shanter,43.781638,-79.304302,Eight Noodles,43.778234,-79.308299,Noodle House
4,Tam O'Shanter,43.781638,-79.304302,Kub Khao,43.780438,-79.299837,Thai Restaurant
5,Tam O'Shanter,43.781638,-79.304302,Little Caesars,43.780563,-79.298624,Pizza Place
6,Tam O'Shanter,43.781638,-79.304302,Popeyes,43.780476,-79.298460,Fried Chicken Joint
7,Tam O'Shanter,43.781638,-79.304302,KFC,43.779440,-79.303371,Fast Food Restaurant
8,Tam O'Shanter,43.781638,-79.304302,Enterprise Rent-A-Car,43.779370,-79.306530,Rental Car Location
9,Tam O'Shanter,43.781638,-79.304302,Gusto Pizza,43.783607,-79.298983,Pizza Place


In [18]:
scarborough_venues.tail(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
74,Cedarbrae,43.773136,-79.239476,The Avenue,43.772437,-79.241962,Lounge
75,Agincourt,43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
76,Agincourt,43.794200,-79.262029,Twilight,43.791999,-79.258584,Lounge
77,Agincourt,43.794200,-79.262029,Subway,43.792823,-79.259681,Sandwich Place
78,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,Kairali,43.754768,-79.277199,Indian Restaurant
79,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,Kim Kim restaurant,43.753833,-79.276611,Chinese Restaurant
80,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,El Pulgarcito,43.754790,-79.277064,Latin American Restaurant
81,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,Karaikudi Chettinad South Indian Restaurant,43.756042,-79.276276,Indian Restaurant
82,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,Big Al's Pet Supercentre,43.759279,-79.278325,Pet Store
83,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,Pho Vietnam,43.757770,-79.278572,Vietnamese Restaurant


In [19]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,3,3,3,3,3,3
"Agincourt North, Milliken",2,2,2,2,2,2
Birch Cliff,4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Cliffcrest, Cliffside",3,3,3,3,3,3
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"Highland Creek, Rouge Hill, Port Union",1,1,1,1,1,1
"Ionview, Kennedy Park",5,5,5,5,5,5


In [20]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 51 uniques categories.


In [21]:

# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Tam O'Shanter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Tam O'Shanter,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Tam O'Shanter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Tam O'Shanter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [22]:
scarb_onehot.shape

(84, 52)

In [23]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000,0.0,0.000,0.000,0.0,0.333333,0.0,0.0,0.00,0.000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.333333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.333333,0.000000,0.0,0.0,0.000,0.0,0.000000
1,"Agincourt North, Milliken",0.0,0.000000,0.000,0.0,0.000,0.000,0.0,0.000000,0.0,0.0,0.00,0.000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.5,0.000000,0.0,0.0,0.5,0.0,0.000000,0.000000,0.0,0.0,0.000,0.0,0.000000
2,Birch Cliff,0.0,0.000000,0.000,0.0,0.000,0.000,0.0,0.000000,0.0,0.0,0.25,0.000,0.000000,0.0,0.25,0.0,0.0,0.0,0.0,0.000,0.25,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.250000,0.0,0.0,0.000,0.0,0.000000
3,Cedarbrae,0.0,0.000000,0.125,0.0,0.125,0.125,0.0,0.000000,0.0,0.0,0.00,0.125,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.125,0.00,0.0,0.125,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.125000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.125,0.0,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000000,0.000,0.0,0.200,0.000,0.0,0.000000,0.2,0.1,0.00,0.000,0.000000,0.0,0.00,0.0,0.0,0.0,0.1,0.000,0.00,0.0,0.000,0.0,0.000000,0.1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.1,0.0,0.0,0.000000,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.1,0.0,0.000,0.0,0.000000
5,"Cliffcrest, Cliffside",0.0,0.333333,0.000,0.0,0.000,0.000,0.0,0.000000,0.0,0.0,0.00,0.000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.333333,0.0,0.0,0.000,0.0,0.000000
6,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000000,0.000,0.0,0.000,0.000,0.0,0.000000,0.0,0.0,0.00,0.000,0.166667,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.000,0.0,0.333333,0.0,0.0,0.0,0.0,0.166667,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.0,0.166667


### Get top 10 venues per neighborhood

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Sandwich Place,Lounge,Breakfast Spot,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Agincourt North, Milliken",Playground,Park,Vietnamese Restaurant,Caribbean Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
2,Birch Cliff,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Caribbean Restaurant,Bakery,Fried Chicken Joint,Lounge,Bank,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Auto Garage,Grocery Store
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Park,Metro Station,Coffee Shop,Grocery Store
5,"Cliffcrest, Cliffside",American Restaurant,Skating Rink,Motel,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Pet Store,Latin American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Bakery,Bank,Hakka Restaurant,Grocery Store,General Entertainment
7,"Highland Creek, Rouge Hill, Port Union",Bar,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
8,"Ionview, Kennedy Park",Hobby Shop,Department Store,Convenience Store,Bus Station,Coffee Shop,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint
9,"Maryvale, Wexford",Middle Eastern Restaurant,Accessories Store,Auto Garage,Sandwich Place,Breakfast Spot,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant


### Run k-means to cluster the neighborhoods into 5 clusters

In [26]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(16)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([0, 4, 0, 0, 0, 0, 0, 3, 0, 0], dtype=int32)

### Include kmeans.labels_ into the original Scarborough dataframe

In [27]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarb_merged

,Unnamed: 0,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Department Store,Convenience Store
1,11,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302,4,Pizza Place,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Rental Car Location,Pharmacy,Noodle House,Chinese Restaurant,Vietnamese Restaurant
2,13,M1W,Scarborough,Steeles West,43.799525,-79.318389,0,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Nail Salon,Thrift / Vintage Store,Japanese Restaurant,Sandwich Place,Pizza Place,Pharmacy,Breakfast Spot
3,32,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,33,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476,0,American Restaurant,Skating Rink,Motel,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,39,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Bar,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
6,42,M1R,Scarborough,"Maryvale, Wexford",43.750071,-79.295849,0,Middle Eastern Restaurant,Accessories Store,Auto Garage,Sandwich Place,Breakfast Spot,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
7,47,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,3,Spa,Playground,Vietnamese Restaurant,Caribbean Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
8,49,M1V,Scarborough,"Agincourt North, Milliken",43.815252,-79.284577,0,Playground,Park,Vietnamese Restaurant,Caribbean Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
9,50,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0,Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Park,Metro Station,Coffee Shop,Grocery Store


### Visualize the clusters in the map

In [28]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine each of the five clusters

In [29]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Department Store,Convenience Store
2,M1W,-79.318389,0,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Nail Salon,Thrift / Vintage Store,Japanese Restaurant,Sandwich Place,Pizza Place,Pharmacy,Breakfast Spot
3,M1X,-79.205636,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1M,-79.239476,0,American Restaurant,Skating Rink,Motel,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,M1C,-79.160497,0,Bar,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
6,M1R,-79.295849,0,Middle Eastern Restaurant,Accessories Store,Auto Garage,Sandwich Place,Breakfast Spot,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
8,M1V,-79.284577,0,Playground,Park,Vietnamese Restaurant,Caribbean Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
9,M1L,-79.284577,0,Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Park,Metro Station,Coffee Shop,Grocery Store
10,M1G,-79.216917,0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Indian Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
13,M1E,-79.188711,0,Electronics Store,Rental Car Location,Pizza Place,Breakfast Spot,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint


In [30]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,M1S,-79.262029,1,Sandwich Place,Lounge,Breakfast Spot,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [31]:

scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,M1K,-79.262029,2,Hobby Shop,Department Store,Convenience Store,Bus Station,Coffee Shop,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint


In [32]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,M1J,-79.239476,3,Spa,Playground,Vietnamese Restaurant,Caribbean Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store


In [33]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1T,-79.304302,4,Pizza Place,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Rental Car Location,Pharmacy,Noodle House,Chinese Restaurant,Vietnamese Restaurant
12,M1N,-79.264848,4,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
